In [2]:
import range_construction
import flop_classifier
import pandas as pd
import numpy as np
import itertools
import random

from treys import Evaluator, Card, Deck

In [3]:
"""
Use monte-carlo simulation to determine an approximate ordering of all hands via preflop equity. 
Law of large numbers: as the number of random samples increases, the average of the samples will approach the true average. Caveat is the growth is asymptotic. 
Meaning that nearing some threshold of hands, the average of the random samples is arguably good enough as an approximation of the true average. There exists a point of diminishing returns,
consider computational cost and accuracy. 

Algorithm will be: 
For each pair of possible starting hands, use Monte-carlo simulations to randomly determine n number of boards. At each street on the board compute the hand that is ahead via treys.evaluator.
This computation will be hand_rank(hand1) - hand_rank(hand2). Add this output to some dataframe for later storage. On the flop classify the flop (determination of these classes below). 
At the end of the simulation, calculate the equity(hand1, hand2) via hand1_wins + hand1_ties / total_boards.  Equity(hand2, hand1) = 1 - equity(hand1, hand2).   
"""

'\nUse monte-carlo simulation to determine an approximate ordering of all hands via preflop equity. \nLaw of large numbers: as the number of random samples increases, the average of the samples will approach the true average. Caveat is the growth is asymptotic. \nMeaning that nearing some threshold of hands, the average of the random samples is arguably good enough as an approximation of the true average. There exists a point of diminishing returns,\nconsider computational cost and accuracy. \n\nAlgorithm will be: \nFor each pair of possible starting hands, use Monte-carlo simulations to randomly determine n number of boards. At each street on the board compute the hand that is ahead via treys.evaluator.\nThis computation will be hand_rank(hand1) - hand_rank(hand2). Add this output to some dataframe for later storage. On the flop classify the flop (determination of these classes below). \nAt the end of the simulation, calculate the equity(hand1, hand2) via hand1_wins + hand1_ties / tot

In [4]:
def assign_placeholder_suits(hand): 
    suits = ['s', 'h', 'd', 'c']

    suit1 = suits[random.randint(0,3)]

    if 's' in hand: return [f"{hand[0]}{suit1}", f"{hand[1]}{suit1}"]

    else:
        suits.remove(suit1)        
        suit2 = suits[random.randint(0,2)]
        return [f"{hand[0]}{suit1}", f"{hand[1]}{suit2}"]

In [5]:
def ensure_unique_suits(hand1, hand2):
    suits = ['s', 'h', 'd', 'c']
    used_cards = set(hand1) 
    
    for i, card2 in enumerate(hand2):
        if card2 in used_cards:
            rank = card2[0]  
            # Find an available suit for this rank
            available_suits = [suit for suit in suits if f"{rank}{suit}" not in used_cards]
            if available_suits:
                new_card = f"{rank}{available_suits[0]}"
                hand2[i] = new_card
                used_cards.add(new_card)
        else:
            used_cards.add(card2)  # Add non-duplicate card to the used set
    
    return hand1, hand2

In [6]:
hand1 = assign_placeholder_suits('AA')
hand2 = assign_placeholder_suits('AA')
ensure_unique_suits(hand1, hand2)

(['As', 'Ad'], ['Ac', 'Ah'])

In [7]:
res = []
equity_results = {}

deck = [
    "2c", "2d", "2h", "2s", "3c", "3d", "3h", "3s", "4c", "4d", "4h", "4s", 
    "5c", "5d", "5h", "5s", "6c", "6d", "6h", "6s", "7c", "7d", "7h", "7s", 
    "8c", "8d", "8h", "8s", "9c", "9d", "9h", "9s", "Tc", "Td", "Th", "Ts", 
    "Jc", "Jd", "Jh", "Js", "Qc", "Qd", "Qh", "Qs", "Kc", "Kd", "Kh", "Ks", 
    "Ac", "Ad", "Ah", "As"
]
evaluator = Evaluator()

# Loop over all possible 2-card combinations
for combo in itertools.combinations(range_construction.hands, 2):
    hand1 = assign_placeholder_suits(combo[0])
    hand2 = assign_placeholder_suits(combo[1])
    hand1, hand2 = ensure_unique_suits(hand1, hand2)

    # Convert these to Treys Card objects
    hand1_cards = [Card.new(c) for c in hand1]
    hand2_cards = [Card.new(c) for c in hand2]

    # Remove these 4 cards from the deck to create the 'remaining deck'
    used_cards = hand1 + hand2
    remaining_deck = [card for card in deck if card not in used_cards]

    hand1_wins = 0
    hand2_wins = 0
    ties = 0

    # Monte-Carlo simulation
    for _ in range(1):
        # Randomly select 3 flop cards
        flop = random.sample(remaining_deck, 3)
        after_flop_deck = [c for c in remaining_deck if c not in flop]
        turn = random.sample(after_flop_deck, 1)
        after_turn_deck = [c for c in after_flop_deck if c not in turn]
        river = random.sample(after_turn_deck, 1)

        # Combine flop + turn + river
        board_as_strings = flop + turn + river
        board_cards = [Card.new(c) for c in board_as_strings] # convert to treys card objects

        # --- Classify the flop (only 3 cards from 'flop') ---
        flop_qualities = []
        flop_qualities.append(flop_classifier.analyze_board_connectivity(flop))
        flop_qualities.append(flop_classifier.analyze_board_suits(flop))
        flop_qualities.append(flop_classifier.analyze_board_pairing(flop))

        if any(card.startswith('A') for card in flop):
            flop_qualities.append('Ace high')
        elif any(card.startswith('K') for card in flop):
            flop_qualities.append('King high')

        dynamic_score = flop_classifier.determine_dynamic_score(flop)

        # Evaluate each 5-card hand
        hand1_rank = evaluator.evaluate(board_cards, hand1_cards)
        hand2_rank = evaluator.evaluate(board_cards, hand2_cards)

        if hand1_rank < hand2_rank:
            hand1_wins += 1
        elif hand1_rank > hand2_rank:
            hand2_wins += 1
        else:
            ties += 1

        res.append({
            'hand1': hand1,
            'hand2': hand2,
            'flop': flop,
            'turn': turn,
            'river': river,
            'flop_quality': flop_qualities,
            'flop_dynamic_score': dynamic_score
        })

    total_simulations = hand1_wins + hand2_wins + ties
    equity_hand1 = (hand1_wins + ties / 2.0) / total_simulations
    equity_hand2 = 1.0 - equity_hand1

    # Record the equity for each distinct 2-card combination
    equity_results.setdefault(combo[0], []).append(equity_hand1)
    equity_results.setdefault(combo[1], []).append(equity_hand2)

# Once the loop finishes, compute average equity per 2-card combo
for hand_str, eq_values in equity_results.items():
    equity_results[hand_str] = np.mean(eq_values)

df_equity = pd.DataFrame(equity_results.items(), columns=['hand', 'equity'])
df_equity.to_csv('preflop_equity.csv', index=False)

df_res = pd.DataFrame(res)
df_res.to_csv('preflop_monte_carlo.csv', index=False)